In [1]:
import asyncio
import json
import requests


from app.utility.helps import Bob

In [2]:
bob = Bob()
settings = bob.get_settings()
headers = bob.get_context()

In [ ]:
api_url = 'https://api.powerbi.com/v1.0/myorg/gateways'
response = requests.get(url=api_url, headers=headers)
results = response.json()
print(response.status_code)
gateways = list()
for gateway in results['value']:
    gateways.append(gateway.get("id"))


In [ ]:
r = list()
for id in gateways:
    response = requests.get(f'https://api.powerbi.com/v1.0/myorg/gateways/{id}/datasources', headers=headers)
    doc_results = response.json()
    r.append(doc_results['value'])   


In [ ]:
gateways = list()

for i in range(0, len(r)):
    if isinstance(r[i], list):
        for j in range(0, len(r[i])):
            gateways.append(r[i][j])
    else:
        gateways.append(r[i])

In [ ]:
gateways

In [ ]:
users = list()

for i in range(0,len(gateways)):
    api_users = f'https://api.powerbi.com/v1.0/myorg/gateways/{gateways[i]["gatewayId"]}/datasources/{gateways[i]["id"]}/users'
    response = requests.get(api_users, headers=headers)
    results = response.json()
    results['value'][0]['datasourceId'] = gateways[i]["id"]
    results['value'][0]['gatewayId'] = gateways[i]["gatewayId"]
    users.append(results['value'][0])
    

## Datasource Connectivity  

This only reports back connectivity to the datasource as an error. Also, you need to use the response.text as the JSON is not populated. A successful connection will report back as HTTP 200 while a failed connection will result in HTTP 400

In [ ]:
## This will be used for a template dictionary for a successful connection to the datasource
status_base = {   
    "datasourceId":"",
    "gatewayId":"",
    "error":
    {
        "code":"success",
        "pbi.error":{ 
            "code":"0",
            "parameters":{},
            "details":[],
            "exceptionCulprit":0
        }
    }
}


In [ ]:
status  = list()
for i in range(0,len(gateways)):
    api_status = f'https://api.powerbi.com/v1.0/myorg/gateways/{gateways[i]["gatewayId"]}/datasources/{gateways[i]["id"]}/status'
    
    response = requests.get(api_status, headers=headers)
    if response.ok:
        # response.json() is actually empty when it is a success
        
        status_base['datasourceId'] = gateways[i]["id"]
        status_base['gatewayId'] = gateways[i]["gatewayId"]
        status_base.get("error").get("pbi.error").get("details").append({"message":"success","detail":response.status_code})    
        status.append(status_base)
    else:
        results = json.loads(response.text)
        results["datasourceId"] = gateways[i]["id"]
        results["gatewayId"] = gateways[i]["gatewayId"]
        results.get("error").get("pbi.error").get("details").append({"message":"cannot connect","detail":response.status_code})    
        status.append(results)


In [ ]:
datasources = list()

for i in range(0,len(gateways)):
    api_datasource = f'https://api.powerbi.com/v1.0/myorg/gateways/{gateways[i]["gatewayId"]}/datasources/{gateways[i]["id"]}'
    response = requests.get(api_datasource, headers=headers)
    results = response.json()
    results.pop('@odata.context')
    datasources.append(results)


In [ ]:
import smtplib
from email.mime.text import MIMEText

sender_email = 'brcampb@microsoft.com'
receiver_email = 'brandonh.campbell@gmail.com'
subject = 'Hello from Python!'
body = 'This is a test email sent from a Python script.'

msg = MIMEText(body)
msg['Subject'] = subject
msg['From'] = sender_email
msg['To'] = receiver_email

# Set up the SMTP server (e.g., Gmail)
with smtplib.SMTP_SSL('smtp.gmail.com', 465) as server:
    server.login(sender_email, 'your_password')
    server.sendmail(sender_email, [receiver_email], msg.as_string())



sender_email = 'your_email@example.com'
receiver_email = 'recipient_email@example.com'
subject = 'Hello from Python!'
body = 'This is a test email sent from a Python script.'

msg = MIMEText(body)
msg['Subject'] = subject
msg['From'] = sender_email
msg['To'] = receiver_email

# Set up the SMTP server (e.g., Outlook)
with smtplib.SMTP('smtp.office365.com', 587) as server:
    server.starttls()
    server.login(sender_email, 'your_password')
    server.sendmail(sender_email, [receiver_email], msg.as_string())


In [ ]:
import smtplib
from email.mime.text import MIMEText

# Set up the SMTP server
smtp_server = 'smtp.gmail.com'
smtp_port = 587
sender_email = 'your_email@example.com'
password = 'your_password'

# Create a message
subject = 'Hello from Python!'
body = 'This is a test email sent from a Python script.'
msg = MIMEText(body)
msg['Subject'] = subject
msg['From'] = sender_email
msg['To'] = 'recipient_email@example.com'

# Send the email
with smtplib.SMTP(smtp_server, smtp_port) as server:
    server.starttls()
    server.login(sender_email, password)
    server.sendmail(sender_email, [msg['To']], msg.as_string())


In [3]:
# rest_api = "admin/capacities/refreshables"
# GET https://api.powerbi.com/v1.0/myorg/admin/groups?$expand=datasets
# htpps://api.powerbi.com/v1.0/myorg/admin/groups?$expand=datasets
rest_api = "https://api.powerbi.com/v1.0/myorg/admin/groups?$expand=datasets&$top=5000"

response = requests.get(url=rest_api, headers=headers)
workspaces = response.json()


In [ ]:
workspaces

In [6]:
for item in workspaces['value']:
    group_id = item['id']
    for dataset in item['datasets']:
        dataset_id = dataset['id']
        print(dataset)

{'id': '6edf8934-3f70-44de-8958-3f96d8e13563', 'name': 'testing', 'addRowsAPIEnabled': False, 'configuredBy': 'brandon.campbell@mngenvmcap084084.onmicrosoft.com', 'isRefreshable': False, 'isEffectiveIdentityRequired': False, 'isEffectiveIdentityRolesRequired': False, 'targetStorageMode': 'Abf', 'createdDate': '2023-02-28T19:48:45.457Z', 'contentProviderType': 'PbixInDirectQueryMode', 'upstreamDatasets': [], 'users': [], 'isInPlaceSharingEnabled': False}
{'id': 'f72a5953-f751-4938-9c70-4c3730c162c5', 'name': 'fabric_data_science', 'addRowsAPIEnabled': False, 'configuredBy': 'brandon.campbell@mngenvmcap084084.onmicrosoft.com', 'isRefreshable': True, 'isEffectiveIdentityRequired': False, 'isEffectiveIdentityRolesRequired': False, 'targetStorageMode': 'Abf', 'createdDate': '2023-05-25T14:02:53.723Z', 'contentProviderType': 'InImportMode', 'upstreamDatasets': [], 'users': [], 'isInPlaceSharingEnabled': False}
{'id': 'ed1d7386-081b-42c3-851b-99d2f0c5e95c', 'name': 'diabetes_ds', 'addRowsAPIE

In [7]:
# GET https://api.powerbi.com/v1.0/myorg/groups/{groupId}/datasets/{datasetId}/refreshes
refresh_history = list()

for item in workspaces['value']:
    group_id = item['id']
    for dataset in item['datasets']:
        dataset_id = dataset['id']
        if dataset['isRefreshable']==True and dataset['addRowsAPIEnabled']==False:
            rest_api = f"https://api.powerbi.com/v1.0/myorg/groups/{group_id}/datasets/{dataset_id}/refreshes"

            response = requests.get(url=rest_api, headers=headers)
            if response.ok:
                results = response.json()
                for result in results['value']:
                    if len(result)>0:
                        refresh_history.append(result)




In [8]:
refresh_history

[{'requestId': '9cbca636-ec64-4ff6-9dfe-9e2e5acc3056',
  'id': 17173345,
  'refreshType': 'Scheduled',
  'startTime': '2024-04-03T06:00:08.19Z',
  'endTime': '2024-04-03T06:00:40.707Z',
  'status': 'Completed',
  'refreshAttempts': [{'attemptId': 1,
    'startTime': '2024-04-03T06:00:08.3148007Z',
    'endTime': '2024-04-03T06:00:38.7694624Z',
    'type': 'Data'},
   {'attemptId': 1,
    'startTime': '2024-04-03T06:00:38.9881692Z',
    'endTime': '2024-04-03T06:00:39.2849977Z',
    'type': 'Query'}]},
 {'requestId': 'de2143d8-20a8-4064-b508-b80d08153074',
  'id': 17140420,
  'refreshType': 'Scheduled',
  'startTime': '2024-04-02T06:00:06.16Z',
  'endTime': '2024-04-02T06:00:34.377Z',
  'status': 'Completed',
  'refreshAttempts': [{'attemptId': 1,
    'startTime': '2024-04-02T06:00:06.2834508Z',
    'endTime': '2024-04-02T06:00:32.721383Z',
    'type': 'Data'},
   {'attemptId': 1,
    'startTime': '2024-04-02T06:00:32.9713595Z',
    'endTime': '2024-04-02T06:00:33.2369361Z',
    'type':

In [12]:
import pandas as pd

fd = pd.DataFrame(refresh_history)
fd.head()

,requestId,id,refreshType,startTime,endTime,status,refreshAttempts,serviceExceptionJson
0,9cbca636-ec64-4ff6-9dfe-9e2e5acc3056,17173345,Scheduled,2024-04-03T06:00:08.19Z,2024-04-03T06:00:40.707Z,Completed,"[{'attemptId': 1, 'startTime': '2024-04-03T06:...",NaN
1,de2143d8-20a8-4064-b508-b80d08153074,17140420,Scheduled,2024-04-02T06:00:06.16Z,2024-04-02T06:00:34.377Z,Completed,"[{'attemptId': 1, 'startTime': '2024-04-02T06:...",NaN
2,70fd003f-a0b9-4b7c-9e9a-c47cee637e6d,17106762,Scheduled,2024-04-01T06:00:08.077Z,2024-04-01T06:00:40.03Z,Completed,"[{'attemptId': 1, 'startTime': '2024-04-01T06:...",NaN
3,03f43547-95de-4607-ba46-39eabfd5ea9c,17072787,Scheduled,2024-03-31T06:00:05.77Z,2024-03-31T06:00:34.04Z,Completed,"[{'attemptId': 1, 'startTime': '2024-03-31T06:...",NaN
4,b63cbdfd-cdaf-4f10-bf92-a5df8e0784fd,17038958,Scheduled,2024-03-30T06:00:03.617Z,2024-03-30T06:00:35.5Z,Completed,"[{'attemptId': 1, 'startTime': '2024-03-30T06:...",NaN


In [57]:
fd[fd['requestId']=='9cbca636-ec64-4ff6-9dfe-9e2e5acc3056']['refreshAttempts'][0]

[{'attemptId': 1,
  'startTime': '2024-04-03T06:00:08.3148007Z',
  'endTime': '2024-04-03T06:00:38.7694624Z',
  'type': 'Data',
  'refreshAttempts': 17173345},
 {'attemptId': 1,
  'startTime': '2024-04-03T06:00:38.9881692Z',
  'endTime': '2024-04-03T06:00:39.2849977Z',
  'type': 'Query',
  'refreshAttempts': 17173345}]

In [78]:
import pandas as pd

def normalize(df:pd.DataFrame(), base2:pd.DataFrame(), json_column='workspace_id'):
    for row in df.itertuples():
        column_number = df.columns.get_loc(json_column)+1
        if isinstance(row[column_number], list) and len(row[column_number])>0:
            for item in row[column_number]:
                # item[id_name] = row.id
                base2 = pd.concat([base2, df, pd.json_normalize(item)])
                base2.drop(json_column, axis=1, inplace=True)
        else:
            if isinstance(row[column_number], dict):
                print("Yes it is a dictionary")
                base2 = pd.concat([base2, df, pd.json_normalize(row[column_number])])
                base2.drop(json_column, axis=1, inplace=True)
    return base2

In [77]:
dd = normalize(fd, pd.DataFrame(), 'refreshAttempts')
dd.head()

,requestId,id,refreshType,startTime,endTime,status,serviceExceptionJson,attemptId,type
0,9cbca636-ec64-4ff6-9dfe-9e2e5acc3056,17173345.0,Scheduled,2024-04-03T06:00:08.19Z,2024-04-03T06:00:40.707Z,Completed,NaN,NaN,NaN
1,de2143d8-20a8-4064-b508-b80d08153074,17140420.0,Scheduled,2024-04-02T06:00:06.16Z,2024-04-02T06:00:34.377Z,Completed,NaN,NaN,NaN
2,70fd003f-a0b9-4b7c-9e9a-c47cee637e6d,17106762.0,Scheduled,2024-04-01T06:00:08.077Z,2024-04-01T06:00:40.03Z,Completed,NaN,NaN,NaN
3,03f43547-95de-4607-ba46-39eabfd5ea9c,17072787.0,Scheduled,2024-03-31T06:00:05.77Z,2024-03-31T06:00:34.04Z,Completed,NaN,NaN,NaN
4,b63cbdfd-cdaf-4f10-bf92-a5df8e0784fd,17038958.0,Scheduled,2024-03-30T06:00:03.617Z,2024-03-30T06:00:35.5Z,Completed,NaN,NaN,NaN


In [79]:
de = normalize(dd, pd.DataFrame(), 'serviceExceptionJson')
de

""


In [ ]:
import json

string_dict = "{'id': '8271b9f0-3ff7-46b5-9f8a-5a3ba59d01b7', 'name': 'FabricMonitor', 'addRowsAPIEnabled': False, 'configuredBy': 'brandon.campbell@mngenvmcap084084.onmicrosoft.com', 'isRefreshable': True, 'isEffectiveIdentityRequired': False, 'isEffectiveIdentityRolesRequired': False, 'targetStorageMode': 'Abf', 'createdDate': '2024-03-27T18:42:49.663Z', 'contentProviderType': 'PbixInImportMode', 'upstreamDatasets': [], 'users': [], 'isInPlaceSharingEnabled': False}"



print(dict_obj)


In [96]:
for row in dd.itertuples():
    if isinstance(row.serviceExceptionJson, list) and len(row.serviceExceptionJson)>0:
        for item in row.serviceExceptionJson:
            print(item)
    else:
        if isinstance(row.serviceExceptionJson, str):
            print("Yes it is a dictionary")
            print(row.serviceExceptionJson)

Yes it is a dictionary
{"errorCode":"ModelRefresh_ShortMessage_ProcessingError","errorDescription":"{\"error\":{\"code\":\"DM_GWPipeline_Client_GatewayUnreachable\",\"pbi.error\":{\"code\":\"DM_GWPipeline_Client_GatewayUnreachable\",\"parameters\":{},\"details\":[],\"exceptionCulprit\":1}}} Table: Catalog - Workspace."}
Yes it is a dictionary
{"errorCode":"ModelRefresh_ShortMessage_ProcessingError","errorDescription":"{\"error\":{\"code\":\"DM_GWPipeline_Client_GatewayUnreachable\",\"pbi.error\":{\"code\":\"DM_GWPipeline_Client_GatewayUnreachable\",\"parameters\":{},\"details\":[],\"exceptionCulprit\":1}}} Table: Catalog - DataSource."}
Yes it is a dictionary
{"errorCode":"ModelRefresh_ShortMessage_ProcessingError","errorDescription":"[ValidateMarkupTags][ccon]Expression.Error: The key didn't match any rows in the table.. Key = [ccon][entity = \"Catalog - Permissions\", version = \"\"][\/ccon]. Table = [ccon]#table({\"entity\", \"dataCategory\", \"Data\", \"Tags\", \"entityName\", \"ve

In [ ]:
api="https://api.powerbi.com/v1.0/myorg/admin/capacities/refreshables?$top=5000"

refreshables = list()

response = requests.get(url=api, headers=headers)
if response.ok:
    results = response.json()
    for result in results['value']:
        refreshables.append(result)

        

In [ ]:
if len(refresh_history)>=len(refreshables):
    print("there is more information in refresh_history than in refreshables")
else:
    print("there is more information in refreshables than in refresh_history")

In [ ]:
refresh_history[0]

In [ ]:
import pandas as pd

df = pd.json_normalize(refreshables)
df.head()

In [ ]:
df2 = pd.json_normalize(refresh_history)

In [ ]:
df2.head()

In [ ]:
df.to_csv('refreshables.csv', index=False)
df2.to_csv('refresh_history.csv', index=False)

In [ ]:
df3 = df2[['requestId','refreshAttempts']]

In [ ]:
import pandas as pd

# Assuming the DataFrame with the column 'refreshAttempts' is named 'df'
df_expanded = pd.json_normalize(df3['refreshAttempts'])

# Merge the expanded DataFrame with the original DataFrame
df_merged = pd.concat([df3.drop('refreshAttempts', axis=1), df_expanded], axis=1)


In [ ]:
df_merged.head()

In [ ]:
df_unpivoted = df_merged.melt(id_vars=['requestId'], var_name='column', value_name='value')
df_unpivoted.rename(columns={'requestId':'requestId','column':'column','value':'refreshAttempts'}, inplace=True)



In [ ]:
df_unpivoted.head()

In [ ]:
import pandas as pd
import json

# Convert JSON column to DataFrame
df_refreshAttempts = pd.json_normalize(df_unpivoted['refreshAttempts'])

# Merge the original DataFrame with the new DataFrame
df_merged = pd.concat([df_unpivoted.drop('refreshAttempts', axis=1), df_refreshAttempts], axis=1)
df_merged.head()

In [ ]:
refresh_history_clean = df_merged[['requestId','attemptId','startTime','endTime','serviceExceptionJson']]
refresh_history_clean.head()

In [ ]:
refresh_history_cleanest = df2.merge(refresh_history_clean, on='requestId', how='left')
refresh_history_cleanest.head()

In [ ]:
refresh_history_cleanest.drop(columns=['refreshAttempts'], inplace=True)

In [ ]:
refresh_history_cleanest.to_csv('refresh_history_cleanest.csv', index=False)